In [1]:
import io

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import os
import csv
import re
from parse import *
import signal
import csv

In [2]:
def parse_html(url, class_=None, text=None):
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'lxml')
    if class_!=None:
        return soup.find_all(class_=class_)
    elif text != None:
        return soup.find_all(text)

In [3]:
def signal_handler(signum, frame):
    raise Exception("Timed out!")


In [5]:
url = 'https://pytorch.org/cppdocs/api/library_root.html'


In [60]:
content = parse_html(url, class_ = 'toctree-wrapper compound')

In [95]:
content[2]('a')

[<a class="reference internal" href="namespace_at__native.html">Namespace at::native</a>,
 <a class="reference internal" href="namespace_at__native.html#classes">Classes</a>,
 <a class="reference internal" href="namespace_at__native.html#functions">Functions</a>,
 <a class="reference internal" href="namespace_at__native.html#unions">Unions</a>]

In [96]:
content[2]('a')[0]

<a class="reference internal" href="namespace_at__native.html">Namespace at::native</a>

In [97]:
content[2]('a')[0].get('href')

'namespace_at__native.html'

In [98]:
content[2]('a')[0].contents

['Namespace at::native']

In [133]:
prefix_save = ['Function', 'Template Function', 'Class', 'Template Class', 'Struct',\
          'Template Struct' ,'Variable', 'Define', 'Typedef', 'Namespace', 'Enum', 'Union']

prefix_check = ['Template Function', 'Template Class', 'Template Struct', \
                'Function', 'Class',  'Struct','Variable', 'Define', 'Typedef', 'Namespace', 'Enum', 'Union']

In [134]:
result = {}
for p in prefix_save:
    result[p] = []

In [135]:
for c in content:
    
    name = c('a')[0].contents[0]
    href = c('a')[0].get('href')
    saved = False
    
    for p in prefix_check:
        if name.startswith(p):
            func_name = name[len(p)+1:]
            result[p].append({'Name':func_name, 'Link':url.replace('library_root.html', href)})
            saved=True
            break
    if not saved:
        print("exception "+ name)

In [137]:
with open('/Users/danning/Desktop/deepflaw/code/pytorch/cpp_api.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(['API_TYPE', 'NAME', 'LINK'])
    for p in prefix_save:
        for rp in result[p]:
            csv_writer.writerow([p, rp['Name'], rp['Link']])